In [109]:
'''Restricted Bolzmann machine'''
'''Currently not working'''

import numpy as np

class RBM():
    
    def __init__(self,n_visible,n_hidden):
        self.weights = np.random.normal(0,0.1,(n_hidden,n_visible))
        self.bias_h = np.random.normal(0,0.1,(n_hidden))
        self.bias_v = np.random.normal(0,0.1,(n_visible))  
        
    def sample_h(self,X,return_probabilities=False):
        z = X @ self.weights.T + self.bias_h # shape of result: samples x n_hidden
        prob = sigmoid(z)
        if (return_probabilities):
            return prob
        else:
            return bernoulli(prob) 

    def sample_v(self,X,return_probabilities=False):
        z = X @ self.weights + self.bias_v # shape of result: samples x n_visible
        prob = sigmoid(z)
        if (return_probabilities):
            return prob
        else:
            return bernoulli(prob) 
    
    def train(self,x0,xk,ph0,phk,learning_rate=0.01):
        self.weights += learning_rate* (x0.T @ ph0 - xk.T @ phk).T
        self.bias_v += learning_rate* (x0 - xk).sum(axis=0)
        self.bias_h += learning_rate* (ph0 - phk).sum(axis=0)
        
def sigmoid(X):
    return 1/(np.exp(-X)+1)    

def bernoulli(X):
    draws = np.random.random(X.shape)
    return (X > draws).astype(int)

### Generate dummy recommender data

In [154]:
n_p = 1000
n_m = 20

# people who like movies of type 0-5
people = np.random.randint(5,size=n_p)
# movies of type 0-5
movies = np.random.randint(5,size=n_m)

def get_like(i):
    return bernoulli((5-abs(movies - people[i]))/5)

ratings = np.zeros((len(people),len(movies)))

for i in range(len(ratings)):
    ratings[i] = get_like(i)

# save real values and randomly assign some to -1
X = ratings
X_copy = X.copy()
for i in range(len(ratings)):
    n_seen = np.random.randint(4,n_m+1)
    not_seen = np.random.choice(np.arange(n_m),n_m-n_seen,replace=False)
    ratings[not_seen] = -1

In [155]:
rbm = RBM(len(movies),8)

epochs = 1000
k = 10
batch_size = 10
X = ratings

for epoch in range(epochs):
    ind = 0
    for batch in range(len(X)//batch_size+1):
        x0 = X[ind:ind+batch_size]
        ind+=batch_size
        if (len(x0)==0):
            break
        ph0 = rbm.sample_h(x0,True)
        # xk when k=0
        xk = x0
        for i in range(k):
            # feed into the machine
            hk = rbm.sample_h(xk)
            xk = rbm.sample_v(hk)
            # do not update values that are -1
            xk[x0==-1] = -1
        phk = rbm.sample_h(xk,True)
        rbm.train(x0,xk,ph0,phk)

In [156]:
a = rbm.sample_v(rbm.sample_h(X))

In [162]:
(X_copy[X==-1] == a[X==-1]).mean()

0.49

In [149]:
p = []
for i in range(20):
    p.append(((((a == X).sum(axis=1))>=i)).mean())
    print(i,p[i])

0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 0.996
11 0.978
12 0.913
13 0.789
14 0.621
15 0.424
16 0.253
17 0.101
18 0.036
19 0.01
